# Jit example

In [ ]:
# imports
import glob
import torch
from IPython.display import Audio
torch.set_num_threads(1)

from utils import (init_jit_model, get_speech_ts,
                  save_audio, read_audio, 
                  state_generator, single_audio_stream)

## Full audio

In [ ]:
def collect_speeches(tss, wav):
    speech_chunks = []
    for i in tss:
        speech_chunks.append(wav[i['start']: i['end']])
    return torch.cat(speech_chunks)

In [ ]:
model = init_jit_model('files/model.jit', 'cpu')

In [ ]:
wav = read_audio('files/en.wav')

In [ ]:
speech_timestamps = get_speech_ts(wav, model, num_steps=4) # get speech timestamps from full audio file

In [ ]:
speech_timestamps

In [ ]:
save_audio('only_speech.wav', collect_speeches(speech_timestamps, wav), 16000) # merge all speech chunks to one audio
Audio('only_speech.wav')

## Single audio stream

In [ ]:
model = init_jit_model('files/model.jit', 'cpu')
wav = 'files/en.wav'

In [ ]:
for i in single_audio_stream(model, wav):
    if i:
        print(i)

## Multiple audio stream

In [ ]:
model = init_jit_model('files/model.jit', 'cpu')

In [ ]:
audios_for_stream = glob.glob('files/*.wav')
len(audios_for_stream) # total 4 audios

In [ ]:
for i in state_generator(model, audios_for_stream, audios_in_stream=2): # 2 audio stream
    if i:
        print(i)

# Onnx example

In [ ]:
# imports
import glob
import torch
from IPython.display import Audio
torch.set_num_threads(1)
import onnxruntime

from utils import (get_speech_ts, save_audio, read_audio, 
                   state_generator, single_audio_stream)

def init_onnx_model(model_path: str):
    return onnxruntime.InferenceSession(model_path)

def validate_onnx(model, inputs):
    with torch.no_grad():
        ort_inputs = {'input': inputs.cpu().numpy()}
        outs = model.run(None, ort_inputs)
        outs = [torch.Tensor(x) for x in outs]
    return outs

## Full audio

In [ ]:
model = init_onnx_model('files/model.onnx')
wav = read_audio('files/en.wav')

In [ ]:
speech_timestamps = get_speech_ts(wav, model, num_steps=4, run_function=validate_onnx) # get speech timestamps from full audio file

In [ ]:
speech_timestamps

In [ ]:
save_audio('only_speech.wav', collect_speeches(speech_timestamps, wav), 16000) # merge all speech chunks to one audio
Audio('only_speech.wav')

## Single audio stream

In [ ]:
model = init_onnx_model('files/model.onnx')
wav = 'files/en.wav'

In [ ]:
for i in single_audio_stream(model, wav, run_function=validate_onnx):
    if i:
        print(i)

## Multiple audio stream

In [ ]:
model = init_onnx_model('files/model.onnx')

In [ ]:
audios_for_stream = glob.glob('files/*.wav')
len(audios_for_stream) # total 4 audios

In [ ]:
for i in state_generator(model, audios_for_stream, audios_in_stream=2, run_function=validate_onnx): # 2 audio stream
    if i:
        print(i)